<a href="https://colab.research.google.com/github/Garima27dec/Emotion-Detection-with-Multiple-Data-Streams/blob/main/BERT_Text_Sentiment_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow_hub as hub

In [2]:
!pip install -U tensorflow-text==2.12.0
import tensorflow_text as text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 45.0 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import numpy as np
import pandas as pd
import cv2
import seaborn as sns
import shutil
import tensorflow as tf
import tensorflow.keras as keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM,Dropout,TimeDistributed, Flatten, Bidirectional, Input
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
import matplotlib.pyplot as plt
from keras.layers import BatchNormalization
from keras.regularizers import l2
from keras.losses import CategoricalCrossentropy
from keras.metrics import CategoricalAccuracy
import re

In [52]:
dataset = pd.read_csv('/content/drive/MyDrive/Emotion_Recognition/Text_ER/tweet_emotions.csv')

In [53]:
dataset.head()

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...


In [54]:
dataset = dataset.drop(['tweet_id'],axis=1)
dataset.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [55]:
print(dataset['sentiment'].unique())

['empty' 'sadness' 'enthusiasm' 'neutral' 'worry' 'surprise' 'love' 'fun'
 'hate' 'happiness' 'boredom' 'relief' 'anger']


In [56]:
dataset['sentiment'].value_counts()

neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: sentiment, dtype: int64

In [57]:
dataset['sentiment'] = dataset['sentiment'].replace(["worry","happiness","sadness","love","surprise","fun","relief","hate","empty","enthusiasm", "boredom", "anger","neutral"], ["Fear","Happy","Sad","Happy","Surprise","Happy","Neutral","Disgust","Sad","Happy","Sad","Angry", "Neutral"])

In [58]:
dataset['sentiment'].value_counts()

Happy       11586
Neutral     10164
Fear         8459
Sad          6171
Surprise     2187
Disgust      1323
Angry         110
Name: sentiment, dtype: int64

In [59]:
labels, features = [],[]

labels = pd.get_dummies(dataset['sentiment']).values

dataset =dataset.drop(['sentiment'], axis=1)

#converting content column in train and val set to text so as to convert any floating point/int content to str
features = dataset.content.astype(str).values

In [60]:
labels

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [61]:
features

array(['@tiffanylue i know  i was listenin to bad habit earlier and i started freakin at his part =[',
       'Layin n bed with a headache  ughhhh...waitin on your call...',
       'Funeral ceremony...gloomy friday...', ...,
       "Happy Mother's Day to all the mommies out there, be you woman or man as long as you're 'momma' to someone this is your day!",
       '@niariley WASSUP BEAUTIFUL!!! FOLLOW ME!!  PEEP OUT MY NEW HIT SINGLES WWW.MYSPACE.COM/IPSOHOT I DEF. WAT U IN THE VIDEO!!',
       '@mopedronin bullet train from tokyo    the gf and i have been visiting japan since thursday  vacation/sightseeing    gaijin godzilla'],
      dtype=object)

In [62]:
X_train,X_val,y_train,y_val = train_test_split(features,labels, test_size=0.25)
print("Input training features and label shape are {},{}".format(X_train.shape,y_train.shape))
print("Input validation features and label shape are {}, {}".format(X_val.shape,y_val.shape))

Input training features and label shape are (30000,),(30000, 7)
Input validation features and label shape are (10000,), (10000, 7)


In [63]:
bert_model_name_encoder = 'bert_en_uncased_L-12_H-768_A-12'
encoder_download_link = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3'
bert_model_name_preprocess = 'bert_en_uncased_L-12_H-768_A-12'
preprocess_download_link = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [64]:
print(f'BERT model selected           : {encoder_download_link}')
print(f'Preprocess model auto-selected: {preprocess_download_link}')

BERT model selected           : https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [65]:
bert_preprocessor = hub.KerasLayer(preprocess_download_link, name='preprocessing')
bert_encoder = hub.KerasLayer(encoder_download_link, name='BERT_encoder')

In [66]:
def classifier_model():
  text_input = Input(shape =(), dtype= tf.string, name='text')
  #preprocesing layer
  preprocessed_text = bert_preprocessor(text_input)
  #passing through encoder layer
  embeddings = bert_encoder(preprocessed_text)
  #fetching the pooled_output from the dictionary of outputs
  output = embeddings['pooled_output']
  output = Dropout(0.2)(output)
  #passing through final dense layer
  output = Dense(7, activation=None, name='classifier_end')(output)
  return Model(text_input, output)
classifier = classifier_model()

In [67]:
classifier.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 preprocessing (KerasLayer)     {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [68]:
loss = CategoricalCrossentropy(from_logits=True)
metrics = CategoricalAccuracy()
optimizer = 'adam'

In [69]:
classifier.compile(optimizer = optimizer, loss=loss, metrics=metrics)

In [70]:
#fitting the model
history = classifier.fit(X_train,y_train, validation_data=(X_val,y_val), epochs=5)

Epoch 1/5
938/938 [==============================] - 493s 512ms/step - loss: 1.6298 - categorical_accuracy: 0.3236 - val_loss: 1.4974 - val_categorical_accuracy: 0.4041
Epoch 2/5
938/938 [==============================] - 485s 518ms/step - loss: 1.5474 - categorical_accuracy: 0.3672 - val_loss: 1.5162 - val_categorical_accuracy: 0.3935
Epoch 3/5
938/938 [==============================] - 485s 518ms/step - loss: 1.5258 - categorical_accuracy: 0.3840 - val_loss: 1.4735 - val_categorical_accuracy: 0.4114
Epoch 4/5
938/938 [==============================] - 483s 515ms/step - loss: 1.5115 - categorical_accuracy: 0.3843 - val_loss: 1.4657 - val_categorical_accuracy: 0.4083
Epoch 5/5
938/938 [==============================] - 484s 516ms/step - loss: 1.5040 - categorical_accuracy: 0.3898 - val_loss: 1.4589 - val_categorical_accuracy: 0.4202


In [71]:
#Saving Model

classifier.save('/content/drive/MyDrive/Emotion_Recognition/Text_ER/BERT_M.h5')
classifier.save_weights('/content/drive/MyDrive/Emotion_Recognition/Text_ER/BERT_W.h5')

In [90]:
test_string1 = ["You look beautiful!"]
test_string2 = ["Today is a sunny day."]
test_string3 = ["This is the last day we see each other."]

def predict(test_string):
  class_prob = classifier.predict(test_string, batch_size=1)[0]
  if np.argmax(class_prob)==0:
    print("Angry")
  elif np.argmax(class_prob)==1:
    print("Disgust")
  elif np.argmax(class_prob)==2:
    print("Fear")
  elif np.argmax(class_prob)==3:
    print("Happy")
  elif np.argmax(class_prob)==4:
    print("Neutral")
  elif np.argmax(class_prob)==5:
    print("Sad")
  else:
    print("Surprise")


predict(test_string1)
predict(test_string2)
predict(test_string3)

1/1 [==============================] - 0s 47ms/step
Happy
1/1 [==============================] - 0s 42ms/step
Happy
1/1 [==============================] - 0s 45ms/step
Fear


In [91]:
#Load model

from keras.models import load_model
model_ter = tf.keras.models.load_model(('/content/drive/MyDrive/Emotion_Recognition/Text_ER/BERT_M.h5'), custom_objects={'KerasLayer':hub.KerasLayer})
model_ter.load_weights('/content/drive/MyDrive/Emotion_Recognition/Text_ER/BERT_W.h5')
print("Loaded model from disk")

Loaded model from disk
